### WINDOW FUNCITON

### RANK OVER

In [ ]:
SELECT *,DENSE_RANK() OVER(ORDER BY Salary DESC) AS rank FROM Employee
#rank this column doesn't come with the table at the same time, which means:
#can not use where 'rank'=1 directly below this query

# 可以先rank，在select subset of the columns
SELECT score, DENSE_RANK() OVER(ORDER BY Score DESC) AS Rank FROM Scores

In [ ]:
SELECT customer_id, product_id, DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY COUNT(order_id) DESC) AS Rank FROM Orders
    GROUP BY customer_id, product_id

In [ ]:
DENSE_RANK() OVER(ORDER BY Salary DESC) #111 2
RANK() OVER(ORDER BY Salary DESC) #111 4
ROW_NUMBER() OVER(ORDER BY Salary DESC) #1234

In [ ]:
PERCENT_RANK() OVER(ORDER BY )

In [ ]:
#PARTITION BY
SELECT e.Name AS Employee, e.Salary AS Salary,d.Name AS Department, 
 DENSE_RANK() OVER(PARTITION BY e.DepartmentId ORDER BY e.Salary DESC) AS rank FROM Employee e

In [ ]:
# agg function in partition by 

WITH CTE AS
    (SELECT*FROM
        (SELECT customer_id, product_id, DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY COUNT(customer_id) DESC) AS Rank FROM Orders
        GROUP BY customer_id, product_id)t
    WHERE t.Rank = 1
    )

### Having vs Where clause

In [ ]:
# having is against(and used after) aggregate function, e.g. sum
# where is filering on raw data

# where-groupby-having
SELECT s.course, AVERAGE(s.score) as avg FROM Student s
WHERE s.age > 18
GROUP BY s.course
HAVING avg < 60

# having average(score)>90

### CASE WHEN

In [ ]:
SELECT
CASE
    WHEN id%2=0 THEN id-1
    WHEN id=(SELECT COUNT(*) FROM seat) THEN id
    ELSE id+1
END id #只对id 这一列操作
,student FROM seat
ORDER BY id
#generate the column when searching at the same time

In [ ]:
SELECT 
CASE 
    WHEN A=B AND B=C THEN 'Equilateral'
    WHEN A+B<=C OR B+C<=A OR A+C<=B THEN 'Not A Triangle'
    WHEN A+B>C AND B+C>A AND A+C>B AND A!=B AND B!=C AND C!=A THEN 'Scalene'
    ELSE 'Isosceles'
END AS Result
FROM TRIANGLES 

### CHANGE WITH DATE FORMAT(2019/9/12)

In [ ]:
#USE DATEDIFF()
SELECT w1.Id FROM Weather w1 LEFT JOIN Weather w2 
ON DATEDIFF(Day,w2.RecordDate,w1.RecordDate)=1 #w1-w2
WHERE w1.Temperature>w2.Temperature

### SELECT TABLE ALIAS

In [2]:
#Format
#SELECT T1.Name FROM (SELECT NAME,RANK()OVER(ORDER BY SCORE DESC) rank FROM TABLE1) AS T1

In [ ]:
SELECT * FROM
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Doctor') AS d,
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Professor') AS p

### CTE: WITH AS ...

In [ ]:
# create a temp table
# be able to do later manipulation on that table

#FORMAT
WITH NewTableOne AS(
SELCT* FROM t
WHERE t.score > 60
    ...
)
SELECT * FROM NewTableOne t1
INNER JOIN NewTableOne t2
ON t1.timestamp = t2.timestamp AND t1.RackNo+t2.Rack = 3

In [ ]:
# two CTE statements
WITH CTE_1 AS
    (SELECT COUNT(DISTINCT(CONCAT(sender_id, send_to_id))) AS fr FROM FriendRequest),
    CTE_2 AS 
    (SELECT COUNT(DISTINCT(CONCAT(requester_id, accepter_id))) AS ra FROM RequestAccepted)

### MULTPLE JOIN

In [ ]:
#Join consecutively
SELECT*FROM a
JOIN b ON a.Id=b.Id 
Join c ON b.Id=c.Id
a--b--c
reference:https://www.interfacett.com/blogs/multiple-joins-work-just-like-single-joins/

In [ ]:
SELECT d.Name,p.Name,s.Name,a.Name FROM
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Doctor') AS d
FULL OUTER JOIN 
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Professor') AS p
ON d.Row=p.Row
FULL OUTER JOIN 
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Singer') AS s
ON p.Row=s.Row
FULL OUTER JOIN 
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Actor') AS a
ON s.Row=a.Row

### PERCEDING

In [ ]:
# SUM previous 10(n) rows to current row--rolling
#  i will have sum(index-3, index-2, index-1) in current row

SUM(DC_CURRENT) OVER(PARTITION BY "RackNo"
                    ORDER BY "Timestamp"
                    ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS "DC_CURRENT_SUM_3"

### MAX-MIN

In [ ]:
SELECT (MAX(POPULATION)-MIN(POPULATION)) a FROM CITY#can be done in one step

### MEDIAN

In [ ]:
SELECT DISTINCT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY LAT_N) OVER () FROM STATION 
#if you want to get percentile in a subgroup you can do partition by in over clause

### REMOVE 0 IN NUMBERS

In [ ]:
SELECT DISTINCT
CEILING(AVG(Salary)-
AVG(CAST(REPLACE(STR(Salary),'0','') AS INT)) )+1
FROM EMPLOYEES
#CEILING ROUND TO LOWEST INTERGER

### TIMESTAMP

In [ ]:
#SELECT id FROM TABLE WHERE YEAR(FROM_UNIXTIME(creation_date)) = 2010
YEAR(CAST(timestamp AS DATE))=2018

### MULTIPLY 1 COLUMN TO ANOTHER& SUBQUERY FROM A TEMPORARY TABLE

In [ ]:
SELECT* FROM
(SELECT (months*salary) AS earnings,DENSE_RANK() OVER(ORDER BY months*salary DESC) rank FROM Employee) AS T

### ROUNDS TO 2 DECIMAL

In [ ]:
33.3333
# DECIMAL -33.00
# ROUND 2 - 33.33

SELECT CONVERT(DECIMAL(10,2),SUM(LAT_N)) FROM table1#123.45

ROUND(AVG(c1.POPULATION),1)#round to nearest integer

### SELECT TWO DIGITS OPERATION

In [ ]:
SELECT ROUND(100.0* t1.immediate_delivery/ t2.all_delivery, 2) AS immediate_percentage FROM 
    (SELECT COUNT(delivery_id) AS immediate_delivery FROM Delivery
    WHERE order_date = customer_pref_delivery_date) t1,
    (SELECT COUNT(delivery_id) AS all_delivery FROM Delivery) t2

### TOP 10 PERCENT

In [ ]:
SELECT NTILE(10) OVER(ORDER BY ID) FROM STATION 